In [1]:
import FedUtils
from model.utils.config import cfg
from model.utils.net_utils import save_checkpoint
import os

## setting

In [2]:
class Arguments():
    def __init__(self):
        self.class_agnostic =False
        self.epochs = 3
        self.lr = 0.001
        self.lr_decay_step = 4
        self.dataset = 'KAIST'
        self.net = 'vgg16'
        self.batch_size = 24
        self.nw = 2
        self.cuda = True
        #self.round = 10
        self.mGPUs=True
        self.optimizer ="sgd"
        self.k=3
        #self.device = th.device("cpu")
        
args = Arguments()

In [3]:
imdb_classes =  ('__background__',  # always index 0
                          'person',
                          'people','cyclist'
                         )

In [4]:
sub_folder = 'wkFedPer_rd'
output_dir =  "models/" + args.net + "/" + args.dataset + "/" + sub_folder



## function

In [5]:

def printModel(model_avg):
    for key in model_avg.module.RCNN_rpn.state_dict():
        print(key)
        print(model_avg.module.RCNN_rpn.state_dict()[key])

    count=0
    for key in model_avg.module.RCNN_base.state_dict():
        print(key)
        
        if count>10:
            print(model_avg.module.RCNN_base.state_dict()[key])
        elif count>15:
            break
        count+=1

In [6]:
def copyBaseAvgPersonal(model_list,ratio_list, avg_model):
    parties = len(model_list)
    model_tmp=[None] * parties
    #optims_tmp=[None] * parties

    for idx, my_model in enumerate(model_list):
        
        model_tmp[idx] = my_model.state_dict()


    for key in model_tmp[0]:    
        #print(key)
        
        if "RCNN_base"  in key: 
            #print("copy base layer from AVG")            
            for i in range(len(model_tmp)):  #copy to each model            
                model_tmp[i][key] = avg_model.state_dict()[key]
            
        else:
            #print("avg other layer")
            model_avg = 0

            for idx, model_tmp_content in enumerate(model_tmp):     # add each model              
                model_avg += ratio_list[idx] * model_tmp_content[key]

            for i in range(len(model_tmp)):  #copy to each model            
                model_tmp[i][key] = model_avg
    for i in range(len(model_list)):    
        model_list[i].load_state_dict(model_tmp[i])
        
    return model_list  #, optims_tmp
    

## load model

In [7]:
for round_i in range(1,10+1):
    ## load avg model
    load_name = os.path.join(output_dir,'faster_rcnn_KAIST_AVG_'+str(round_i)+'.pth')
    model_avg, optimizer, start_round =FedUtils.load_model(imdb_classes, load_name, args, cfg)

## load scene models
    model_path = [
        #output_dir+'/faster_rcnn_KAIST_campus_'+str(round_i)+'_3_718.pth',
        output_dir+'/faster_rcnn_KAIST_road_'+str(round_i)+'_3_513.pth' ,
        output_dir+'/faster_rcnn_KAIST_downtown_'+str(round_i)+'_3_566.pth'
        ]

    parties = len(model_path)


    model_list=[None] * parties
    for i in range(parties):
        model_list[i],optimizer, start_round = FedUtils.load_model(imdb_classes,model_path[i], args,cfg)
    
    
    ratio_list =  [1] * parties 
    ratio_list = [x / parties for x in ratio_list]    
    # copy base layer and average personal layer                             
    model_list = copyBaseAvgPersonal(model_list,ratio_list, model_avg)            
                             
# save model
    save_name = os.path.join(output_dir, 'faster_rcnn_KAIST_AVG_{}_n.pth'.format(round_i))
    save_checkpoint({
      #'session': 1,
      'round': round_i,
      'model':  model_list[0].module.state_dict() if args.mGPUs else model_list[0].state_dict(), 
      'optimizer': optimizer.state_dict(),
      'pooling_mode': cfg.POOLING_MODE,
      'class_agnostic': args.class_agnostic,
    }, save_name)                             

Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained w